In [9]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [10]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "List of months that have 30 days"}
  ]
)
print(response.choices[0].message.content)

{
  "months": [
    "April",
    "June",
    "September",
    "November"
  ]
}


# Parse JSON - Convert from JSON to Python

https://www.w3schools.com/python/python_json.asp

In [14]:
import json

# parse response:
obj: dict[str, list[str]] = json.loads(response.choices[0].message.content)

# the result is a Python dictionary:
print(obj["months"][0])

April


# Function Calling


In [11]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location:str, unit:str="fahrenheit")->str:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [12]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",

            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    
                    "properties": {

                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },

                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},

                    },

                    "required": ["location"],
                },
            },

        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        # display("* Messages: ",messages)
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

In [15]:
run_conversation("What's the weather like in San Francisco, Tokyo, and Paris?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_PD515hIUoakqmor89T7B6CSf', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_xbXWm4r4FMoIfwrUmFLS9YY1', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_HCKAXGgYGPG2OzM6L4G3OwId', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_PD515hIUoakqmor89T7B6CSf', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_xbXWm4r4FMoIfwrUmFLS9YY1', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_HCKAXGgYGPG2OzM6L4G3OwId', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Messages: '

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PD515hIUoakqmor89T7B6CSf', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_xbXWm4r4FMoIfwrUmFLS9YY1', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_HCKAXGgYGPG2OzM6L4G3OwId', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')])]

'* Second Request Messages: '

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PD515hIUoakqmor89T7B6CSf', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_xbXWm4r4FMoIfwrUmFLS9YY1', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_HCKAXGgYGPG2OzM6L4G3OwId', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_PD515hIUoakqmor89T7B6CSf',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_xbXWm4r4FMoIfwrUmF

* Second Response:  {'id': 'chatcmpl-8UY4x5BvB9l986H4LbjQszykDSP7U', 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Currently in San Francisco, the weather is 72°F. In Tokyo, it is 10°C, and in Paris, it is 22°C.', role='assistant', function_call=None, tool_calls=None))], 'created': 1702291531, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_eeff13170a', 'usage': CompletionUsage(completion_tokens=31, prompt_tokens=175, total_tokens=206)}


'Currently in San Francisco, the weather is 72°F. In Tokyo, it is 10°C, and in Paris, it is 22°C.'

If you want to force the model to call a specific function you can do so by setting tool_choice with a specific function name. You can also force the model to generate a user-facing message by setting tool_choice: "none"(means k jo function app ney banaye h woh call nahi hoghai). Note that the default behavior (tool_choice: "auto") is for the model to decide on its own whether to call a function and if so which function to call.(means k uski marzi h k aap k function ko call kray ya phir apnay)